In [1]:
import pyodbc
import pandas as pd
import datetime

from Activo import *
from Accion import *
from Bono import *
from Derivado import *
from DerivadosTipos.DerivadosSCC import *
from DerivadosTipos.DerivadosFWD import *

from Cartera import *

In [2]:
server = '172.16.1.38'
username = 'sa'
password = 'qwerty123'
driver = '{ODBC Driver 17 for SQL Server}'
cn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';UID=' + username + ';PWD=' + password)

In [3]:
def cambiar_fecha(fecha):

    fechas = fecha.split("-")
    fechas = fechas[2].split(" ")[0] + "/" + fechas[1] + "/" + fechas[0]
    return fechas

In [4]:
def derivados_dBDerivados(cantidad, cn):

    derivados = "SELECT TOP ("+ str(cantidad) +") * FROM [dbDerivados].[dbo].[TdCarteraDerivados_V2] WHERE Tipo = 'FWD' AND MonedaActivo <> MonedaPasivo"
    derivados = pd.read_sql(derivados, cn)

    derivado = pd.DataFrame()
    dev = []

    for i in range(cantidad):

        info_derivado = dict()
        info_derivado["Tipo"] = derivados["Tipo"][i]
        info_derivado["ID_Key"] = derivados["ID_Key"][i]
        info_derivado["Administradora"] = derivados["Administradora"][i]
        info_derivado["Fondo"] = derivados["Fondo"][i]
        info_derivado["Contraparte"] = derivados["Contraparte"][i]
        info_derivado["ID"] = int(derivados["ID"][i])
        info_derivado["Nemotecnico"] = derivados["Nemotecnico"][i]
        info_derivado["Mercado"] = derivados["Mercado"][i]     
        fecha = datetime.date(2018, 5, 14)
        hora = '1700'
        info_derivado["FechaEfectiva"] = cambiar_fecha(str(derivados["FechaEfectiva"][i]))
        info_derivado["FechaVenc"] = cambiar_fecha(str(derivados["FechaVenc"][i]))
        info_derivado["AjusteFeriados"] = derivados["AjusteFeriados"][i]
        info_derivado["NocionalActivo"] = derivados["NocionalActivo"][i]
        info_derivado["NocionalPasivo"] = derivados["NocionalPasivo"][i]

        info_derivado["MonedaActivo"] = derivados["MonedaActivo"][i]
        info_derivado["MonedaPasivo"] = derivados["MonedaPasivo"][i]

        info_derivado["MonedaBase"] = derivados["MonedaBase"][i]
        info_derivado["TipoTasaActivo"] = derivados["TipoTasaActivo"][i]
        info_derivado["TipoTasaPasivo"] = derivados["TipoTasaPasivo"][i]
        info_derivado["TasaActivo"] = derivados["TasaActivo"][i]
        info_derivado["TasaPasivo"] = derivados["TasaPasivo"][i]
        info_derivado["FrecuenciaActivo"] = derivados["FrecuenciaActivo"][i]
        info_derivado["FrecuenciaPasivo"] = info_derivado["FrecuenciaActivo"]

        info1 = pd.DataFrame([info_derivado])
        derivado_info = DerivadosFWD(fecha, hora, info1, cn)

        dev.append(derivado_info)

    derivado["Derivado"] = dev
    return derivado


In [5]:
derivado = derivados_dBDerivados(1, cn)
derivado["Derivado"][0].genera_flujos()
derivado["Derivado"][0].valoriza_flujos()
print(derivado["Derivado"][0].flujos_valorizados[["ID","ActivoPasivo", "Fecha", "FechaFixing", "FechaFlujo", "FechaPago", "Flujo", "ValorPresenteMonFlujo", "Moneda", "MonedaBase"]])


     ID ActivoPasivo       Fecha FechaFixing  FechaFlujo   FechaPago  \
0  1364            1  2018-05-14  2019-08-08  2019-08-08  2019-08-08   
1  1364           -1  2018-05-14  2019-08-08  2019-08-08  2019-08-08   

       Flujo  ValorPresenteMonFlujo Moneda MonedaBase  
0  2576025.0           2.489592e+06    USD        USD  
1  2250000.0           2.257327e+06    EUR        USD  


In [6]:
cartera = Cartera(pd.DataFrame(), pd.DataFrame(), derivado, 'CLP', datetime.date(2019,2,1), cn)

[0.08333333333333333, 0.25, 0.5, 1.0, 2, 3, 4, 5, 7, 9, 10, 15, 20, 30]
[0.5, 1.0]
2019-08-08
Empty DataFrame
Columns: []
Index: []


KeyError: 0

In [ ]:
cartera.set_hist_ret_vol_totales()

In [ ]:
cartera.get_historicos_totales()

In [ ]:
cartera.get_retornos_totales()

In [ ]:
cartera.get_correlacion()

In [ ]:
cartera.set_volatilidad_cartera()
volatilidad = cartera.get_volatilidad_cartera()
volatilidad